In [6]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Method comparison of us and ucss
us: UniversalSpline from scipy

uccs: UnivariateCubicSmoothingSpline from csaps

In [7]:
import ipywidgets
import numpy
import pandas
import pathlib
import matplotlib.cm
from matplotlib import pyplot
import scipy
import csaps

import bletl
import bletl_analysis

### Parse the raw data file

In [8]:
filepath = pathlib.Path(r'..\bletl\tests\data\BL1\NT_1400_BS20BS10_15min_20160222_151645.csv')   
bldata = bletl.parse(filepath, lot_number=1515, temp=30)

### Plot the data
Now we can visualize the us and ucss. The first subplot shows the splines and the datapoints. The second one shows the derivatives.
If filterset 'BS20' is selected the function 'get_mue' from bletl_analysis is calculated in a third subplot.


In [12]:
def plot_well(well:str, filterset:str, last_cycle:int=None):
    x, y = bldata[filterset].get_timeseries(well)
    x = x[:last_cycle]
    y = y[:last_cycle]

    #get spline us
    spline_us = bletl_analysis.get_crossvalidate_spline(x, y, method="us")
    sp_y_us = spline_us(x)
    
    #get spline ucss
    spline_ucss = bletl_analysis.get_crossvalidate_spline(x, y, method="ucss")
    sp_y_ucss = spline_ucss(x)
    
    #derivative us
    func_us = spline_us.derivative(1)
    us_val = func_us(x)

    #derivative ucss
    func_ucss = spline_ucss.derivative(1)
    ucss_val = func_ucss(x)
    
    #plotting
    fig, (up, middle, down) = pyplot.subplots(dpi=140, nrows=3, sharex='col', figsize=(12,14))
    up.scatter(x, y, s=.4, label='data')
    up.plot(x, spline_ucss(x), label='ucss', linewidth=0.5)
    up.plot(x, spline_us(x), label='us', linewidth=0.5)
    
    middle.axhline(0, color='gray', linewidth=1, linestyle=':')
    middle.plot(x, ucss_val, label='ucss derivative')
    middle.plot(x, us_val, label='us derivative')
    
    #calculates get_mue only for filterset BS20
    if filterset == 'BS20':    
        #get_mue us
        mue_us = bletl_analysis.get_mue(bldata[filterset],wells=[well], method="us", last_cycle=last_cycle)
        df_mue_us = mue_us.get_unified_dataframe()
        df_mue_us.reset_index(inplace=True)
        us_value = numpy.array(df_mue_us[well], dtype=pandas.Series)
        time = numpy.array(df_mue_us["time in h"], dtype=pandas.Series)

        #get_mue ucss
        mue_ucss = bletl_analysis.get_mue(bldata[filterset],wells=[well], method="ucss", last_cycle=last_cycle)
        df_mue_ucss = mue_ucss.get_unified_dataframe()
        df_mue_ucss.reset_index(inplace=True)
        ucss_value = numpy.array(df_mue_ucss[well], dtype=pandas.Series)
        
        #plot
        down.axhline(0, color='gray', linewidth=1, linestyle=':')
        down.plot(time, ucss_value, label='UCSS derivative')
        down.plot(time, us_value, label='US derivative')
        down.set_xlabel('time   [h]')
        down.set_ylabel('get_mue')
        down.set_ylim(-0.5, 1)
    else:
        down.set_visible(False)
    
    #formatting
    middle.set_ylabel('derivative')
    up.set_ylabel(filterset)
    up.legend()

    pyplot.show()
    return

ipywidgets.interact(
    plot_well,
    well=list(bldata['pH'].time.columns),
    filterset=list(bldata.keys()),
    last_cycle=range(10, len(bldata['pH'].time)+1)
);

interactive(children=(Dropdown(description='well', options=('A01', 'A02', 'A03', 'A04', 'A05', 'A06', 'A07', '…